In [14]:
import json

with open("workflows-bioschemas-dump (1).jsonld", "r") as f:
    data = json.load(f)

docs = []
for item in data:
    text = f"Workflow name: {item.get('name')}\n"
    text += f"Platform: {item.get('platform')}\n"
    creator = item.get("creator")
    if isinstance(creator, dict):
        creator_name = creator.get("name")
    else:
        creator_name = creator
    text += f"Creator: {creator_name}\n"
    text += f"Description: {item.get('description')}\n"
    docs.append({"id": item["@id"], "text": text})

In [ ]:
from rdflib import Graph

g = Graph()
g.parse("workflowhub-graph.ttl", format="turtle")  


for subj, pred, obj in g:
    print(f"{subj} --[{pred}]--> {obj}") 

In [ ]:
from collections import defaultdict

entity_info = defaultdict(list)


for s, p, o in g:
    entity_info[s].append((p, o))

In [4]:
rag_docs = []

for subject, props in entity_info.items():
    lines = [f"Entity: {subject}"]
    for p, o in props:
        lines.append(f"  {p.split('/')[-1]} → {o}")
    doc_text = "\n".join(lines)
    rag_docs.append({"id": str(subject), "text": doc_text})

In [ ]:
print(rag_docs[2]["text"])

In [ ]:
entities = set(g.subjects())

for entity in entities:
    print(f"\nEntity: {entity}")
    for p, o in g.predicate_objects(subject=entity):
        print(f"  {p.split('/')[-1]} → {o}")

In [15]:
from rdflib import Graph

g = Graph()
g.parse("workflowhub-graph.ttl", format="turtle")

def enrich_with_relationships(uri):
    lines = []
    for p, o in g.predicate_objects(uri):
        lines.append(f"{p.split('/')[-1]} → {o}")
    return "\n".join(lines)

for doc in docs:
    relationships = enrich_with_relationships(doc["id"])
    doc["text"] += "\nRelationships:\n" + relationships


In [ ]:
print(len(docs), "documents processed.")
for i, doc in enumerate(docs[:3]):
    relationships = enrich_with_relationships(doc["id"])
    doc["text"] += "\nRelationships:\n" + relationships
    print(f"\n==== Document {i+1} ====")
    print(doc["text"])